# Trying to read AFNI BRIK & HEAD files in Python.

Here is a description of the header contents:
    https://afni.nimh.nih.gov/pub/dist/doc/program_help/README.attributes.html
    
Stuff for reading BRIK files, in Matlab there is a tool: https://github.com/PrincetonUniversity/princeton-mvpa-toolbox/blob/master/afni_matlab/BrikLoad.m

In [87]:
import re
import numpy as np

In [55]:
fname = "anat_final.jamie+tlrc"

In [77]:
def read_header(fname):
    """ 
    Reads AFNI header file. That is, reads the HEAD from a BRIK/HEAD file pair.
    
    Arguments
    fname : the filename to be read (if it doesn't end in .HEAD this will be appended)
    
    Returns
    dict of key-values representing the header contents
    """ 
    
    if fname.endswith('.'):
        fname+="HEAD"
    if not fname.endswith('.HEAD'):
        fname+=".HEAD"
    
    # Read the file contents
    headf = open(fname,'r').read()
    
    # Now parse the contents
    remainder = headf[:]

    # No, this is not an insult, it's a pattern that matches the beginning of a chunk,
    # i.e. type=something, name=something_else, count=a_number
    chunkhead = re.compile(r'type\s*=\s*(integer|float|string)-attribute\s*name\s*=\s*(\w+)\s*count\s*=\s*(\d+)')

    header = {}
    types = {}

    m = re.search(chunkhead,remainder)

    type_regexp = {
        "integer":"[+-]?[0-9]",
        "float":"[-+]?(\d+([.,]\d*)?|[.,]\d+)([eE][-+]?\d+)?" # source: http://stackoverflow.com/questions/4703390/how-to-extract-a-floating-number-from-a-string
    }

    # While there is another chunk to be found...
    while m:

        # Parse the type/name/count fields
        endpos = m.end()
        (tp,name,count) = m.groups()
        count = int(count)
        header[name]=count
        types[name]=tp
        #print(m.start(),tp,name,count)

        # The rest of the file...
        remainder = remainder[endpos:]

        # Now read the actual contents
        if tp=='integer' or tp=="float":
            # Set up a regexp that will capture the next "count" ints
            contents = re.match(r'(\s*(%s)+){%i}'%(type_regexp[tp],count),remainder)
            cast = int if tp=="integer" else float
            if contents:
                values = [ cast(i) for i in contents.group().split() ]
                header[name]=values if count>1 else values[0]
            else:
                raise ValueError("Failed to parse contents for %s"%name)
                
            remainder = remainder[contents.end():]

        elif tp=="string":
            contents = re.match(r'\s*\'(.{%i})~'%(count-1),remainder,re.DOTALL)
            if contents:
                header[name]=contents.group(1)
            else:
                raise ValueError("Failed to parse contents for %s"%name)
                
            remainder = remainder[contents.end():]
            
        else:
            raise ValueError("Unknown data type %s for %s"%(tp,name))

            
        # Set up for the next iteration of the loop
        m = re.search(chunkhead,remainder)
        
    return header #,types

In [78]:
header = read_header('anat_final.jamie+tlrc')

In [79]:
header

{'BRICK_FLOAT_FACS': 0.0,
 'BRICK_KEYWORDS': '',
 'BRICK_LABS': '#0',
 'BRICK_STATS': [0.0, 635.0],
 'BRICK_TYPES': 1,
 'BYTEORDER_STRING': 'LSB_FIRST',
 'DATASET_DIMENSIONS': [161, 191, 151, 0, 0],
 'DATASET_NAME': 'zyxt',
 'DATASET_RANK': [3, 1, 0, 0, 0, 0, 0, 0],
 'DELTA': [1.0, 1.0, 1.0],
 'HISTORY_NOTE': "[floris@colossus: Fri Feb  3 15:53:51 2017] ===================================\\n[floris@colossus: Fri Feb  3 15:53:51 2017] === History of inputs to 3dcalc ===\\n[floris@colossus: Fri Feb  3 15:53:51 2017] === Input a:\\n[floris@colossus: Fri Feb  3 15:53:51 2017] [floris@colossus: Fri Feb  3 15:53:21 2017] ===================================\\n[floris@colossus: Fri Feb  3 15:53:21 2017] === History of inputs to 3dcalc ===\\n[floris@colossus: Fri Feb  3 15:53:21 2017] === Input a:\\n[floris@colossus: Fri Feb  3 15:53:21 2017] [floris@colossus: Fri Feb  3 15:52:41 2017] 3dSkullStrip -orig_vol -input ./__tt_o20170126_092539MPRAGEADNIiPAT2s100a1001+orig -prefix ./__tt_o20170126_09

In [121]:
def read_brik(fname,header):
    """ Reads BRIK file. Presupposes that we have parsed the .HEAD file and 
    supplied at least the relevant portion of it.
    
    Arguments
    fname : the filename to be read (.BRIK will be added if necessary)
    header : the header information, a set of key-values
    
    Returns
    array containing the data
    """
    
    if fname.endswith('.'):
        fname+="BRIK"
    if not fname.endswith('.BRIK'):
        fname+=".BRIK"
        
    nslices=header["DATASET_DIMENSIONS"][2]
    nframes=header["DATASET_RANK"][1]
    numpix=header["DATASET_DIMENSIONS"][0]*header["DATASET_DIMENSIONS"][1]
    
    n = numpix*nslices*nframes #(Info.DATASET_DIMENSIONS(1) .* Info.DATASET_DIMENSIONS(2) .* Info.DATASET_DIMENSIONS(3) .* Info.DATASET_RANK(2)
    
    # TODO: Use byte order in there
    # TODO:
    
    V = np.fromfile(fname, dtype='d',count=n)
    #V = fread(fidBRIK, n) , [Opt.OutPrecision,typestr]);
    V = np.reshape(V, (header["DATASET_DIMENSIONS"][0], 
                       header["DATASET_DIMENSIONS"][1],
                       nslices,
                       nframes
                       ))
    
    return V

In [122]:
header["DATASET_DIMENSIONS"],header["DATASET_RANK"],header["BRICK_FLOAT_FACS"]

([161, 191, 151, 0, 0], [3, 1, 0, 0, 0, 0, 0, 0], 0.0)

In [123]:
read_brik(fname,header)

ValueError: total size of new array must be unchanged

In [124]:
    nslices=header["DATASET_DIMENSIONS"][2]
    nframes=header["DATASET_RANK"][1]
    numpix=header["DATASET_DIMENSIONS"][0]*header["DATASET_DIMENSIONS"][1]
    


In [125]:
 n = numpix*nslices*nframes

In [126]:
nslices,nframes,numpix

(151, 1, 30751)

In [127]:
 V = np.fromfile(fname+".BRIK", dtype='<f4',count=n)

In [119]:
V.shape

(2321700,)

In [120]:
n

4643401